In [89]:
import Test.QuickCheck
import Control.Applicative
import Data.List (elemIndex)
import Data.Monoid

<h4>Tupley Type</h4>

In [3]:
data Tupley a b = Tupley {first:: a, second:: b} deriving (Eq, Show)

instance (Arbitrary a, Arbitrary b) => Arbitrary (Tupley a b) where
    arbitrary = do
        a <- arbitrary
        b <- arbitrary
        return (Tupley a b)

In [4]:
instance (Semigroup a, Semigroup b) => Semigroup (Tupley a b) where
    (<>) (Tupley a b) (Tupley a' b') = Tupley (a <> a') (b <> b')
    
instance (Monoid a, Monoid b) => Monoid (Tupley a b) where
    mempty = Tupley mempty mempty
    mappend = (<>)

In [9]:
instance Functor (Tupley a) where
    fmap f (Tupley a b) = Tupley a (f b)

In [11]:
instance Monoid a => Applicative (Tupley a) where
    pure = Tupley mempty
    (Tupley u f) <*> (Tupley v x) = Tupley (u `mappend` v)  (f x)

<h4>TypeCheck Exercises</h4>

In [22]:
added :: Maybe Integer
added = fmap (+3) (lookup 3 $ zip [1, 2, 3] [4, 5, 6])
added == Just 9

True

In [24]:
y :: Maybe Integer
y = lookup 3 $ zip [1, 2, 3] [4, 5, 6]

z :: Maybe Integer
z = lookup 2 $ zip [1, 2, 3] [4, 5, 6]

tupled :: Maybe (Integer, Integer)
tupled = (,) <$> y <*> z

tupled == Just (6, 5)

True

In [28]:
x :: Maybe Int
x = elemIndex 3 [1, 2, 3, 4, 5]

y :: Maybe Int
y = elemIndex 4 [1, 2, 3, 4, 5]

max' :: Int -> Int -> Int
max' = max

maxed :: Maybe Int
maxed = liftA2 max' x y

maxed == Just 3

True

In [53]:
xs = [1, 2, 3]
ys = [4, 5, 6]

x :: Maybe Integer
x = lookup 3 $ zip xs ys

y :: Maybe Integer
y = lookup 2 $ zip xs ys

summed :: Maybe Integer
summed = pure sum <*> ((,) <$> x <*> y)

summed == Just 5

True

<h4>Identity</h4>

In [68]:
newtype Identity a = Identity a deriving (Eq, Ord, Show)

In [69]:
instance Functor Identity where
    fmap f (Identity a) = Identity (f a)
    
instance Applicative Identity where
    pure = Identity
    (<*>) (Identity f) (Identity a) = Identity (f a)

In [70]:
xs = [1, 2, 3]
ys = [9, 9, 9]

const <$> xs <*> ys

[1,1,1,2,2,2,3,3,3]

In [71]:
const <$> Identity xs <*> Identity ys

Identity [1,2,3]

<h4>Constant</h4>

In [78]:
newtype Constant a b = Constant { getConstant :: a } deriving (Eq, Ord, Show)

In [79]:
instance Functor (Constant a) where
    fmap _ (Constant a) = Constant a

In [80]:
instance Semigroup a => Semigroup (Constant a b) where
    (<>) (Constant a) (Constant a') = Constant (a <> a')

instance Monoid a => Monoid (Constant a b) where
    mempty = Constant mempty
    mappend = (<>)

In [86]:
instance Monoid a => Applicative (Constant a) where
    pure = mempty
    (<*>) (Constant a) (Constant a') = Constant (a <> a')

In [91]:
f = Constant (Sum 1)
g = Constant (Sum 2)

In [92]:
f <*> g

Constant {getConstant = Sum {getSum = 3}}

<h4>Safe Constructors Name</h4>

In [97]:
validateLength :: Int -> String -> Maybe String
validateLength maxLen s = if (length s > maxLen) then Nothing else Just s

In [98]:
newtype Name = Name String deriving (Eq, Show)

newtype Address = Address String deriving (Eq, Show)

In [100]:
mkName :: String -> Maybe Name
mkName s = fmap Name (validateLength 32 s)

In [103]:
mkAddress :: String -> Maybe Address
mkAddress a = fmap Address (validateLength 100 a)

In [107]:
data Person = Person Name Address deriving (Eq, Show)

In [109]:
mkPerson :: String -> String -> Maybe Person
mkPerson n a = Person <$> mkName n <*> mkAddress a

In [113]:
mkPerson "Rohit" "234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890"

Just (Person (Name "Rohit") (Address "234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890"))

<h4>Safe Constructors Cow</h4>

In [115]:
data Cow = Cow { name:: String, age:: Int, weight:: Int } deriving (Eq, Show)

In [116]:
noEmpty :: String -> Maybe String
noEmpty "" = Nothing
noEmpty str = Just str

In [118]:
noNegative :: Int -> Maybe Int
noNegative n 
    | n >= 0     = Just n
    | otherwise  = Nothing

In [119]:
cowConstructor :: String -> Int -> Int -> Maybe Cow
cowConstructor n a w = Cow <$> noEmpty n <*> noNegative a <*> noNegative w 

In [121]:
cowConstructor "Rohit" (-1) 2

Nothing